## Install the Spanner Python API


In [ ]:
! pip install --upgrade google-cloud-spanner

## Set the following variables 

You at least need to change the Project ID variable to your project's ID. You can change the other variables if you like. 

In [ ]:
project_id = 'your-project-id-here'
instance_id = 'spanner-postgres'

processing_units = 100
database_id = 'pets-postgres'

OPERATION_TIMEOUT_SECONDS = 240

!gcloud services enable spanner.googleapis.com 
print("Spanner Enabled")

print("Done")

## Create a Spanner instance

In [ ]:
import time
from google.cloud import spanner

def create_instance_with_processing_units(instance_id, processing_units):
    """Creates an instance."""
    spanner_client = spanner.Client(project=project_id)

    config_name = "{}/instanceConfigs/regional-us-central1".format(
        spanner_client.project_name
    )

    instance = spanner_client.instance(
        instance_id,
        configuration_name=config_name,
        display_name=instance_id,
        processing_units=processing_units,
    )

    # create() returns a long-running operation
    operation = instance.create()

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print(
        "Created instance {} with {} processing units".format(
            instance_id, instance.processing_units
        )
    )

# Call the function
create_instance_with_processing_units(instance_id, processing_units)

## Create the Pets database

Note: The parameter database_dialect-2 makes this a PostgreSQL database. 

In [ ]:
def create_database(instance_id, database_id):
    """Creates a database and tables for sample data."""
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)

    database = instance.database(
        database_id, database_dialect=2
    )

    # create() returns a long-running operation
    operation = database.create()

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print("Created database {} on instance {}".format(database_id, instance_id))

# Call the function
create_database(instance_id, database_id)


## The following function runs DDL statements to build the database schema

The parameter to the update_ddl() function is an array. You can run one or more DDL commands. 

In [ ]:
def run_ddl_statement(instance_id, database_id, ddl):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    operation = database.update_ddl(
        [ddl]
    )

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print("Ran statement: {}".format(ddl))


## Create the Owners table

In [ ]:
ddl = """CREATE TABLE Owners (
                  OwnerID VARCHAR(36) PRIMARY KEY,
                  OwnerName VARCHAR(1024) NOT NULL
               )"""

run_ddl_statement(instance_id, database_id, ddl)

## Create the Pets table

Note: the Pets table is interleaved with the Owners table. This means each pet is stored with its owner. The primary key of the Pets table uses both OwnerID and PetID. 

In [ ]:
ddl = """CREATE TABLE Pets (
                  OwnerID VARCHAR(36) NOT NULL, 
                  PetID VARCHAR(36) NOT NULL,     
                  PetType VARCHAR(1024) NOT NULL,
                  PetName VARCHAR(1024) NOT NULL,
                  Breed VARCHAR(1024) NOT NULL,
                  PRIMARY KEY (OwnerID, PetID)
              ) 
              INTERLEAVE IN PARENT Owners ON DELETE CASCADE"""



run_ddl_statement(instance_id, database_id, ddl)

## Display the database schema

In [ ]:
def get_database_ddl(instance_id, database_id):
    """Gets the database DDL statements."""
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)
    ddl = spanner_client.database_admin_api.get_database_ddl(database=database.name)

    print("Retrieved database DDL for {}".format(database_id))

    for statement in ddl.statements:
        print(statement)


get_database_ddl(instance_id, database_id)

## Add some test records

In [ ]:
import uuid

def insert_data(instance_id, database_id):
    doug_id = str(uuid.uuid4())
    john_id = str(uuid.uuid4())
    sue_id = str(uuid.uuid4())

    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    with database.batch() as batch:
        batch.insert(
            table="Owners",
            columns=("OwnerID", "OwnerName"),
            values=[
                (doug_id, u"Doug"),
                (john_id, u"John"),
                (sue_id, u"Sue"),
            ],
        )

        batch.insert(
            table="Pets",
            columns=("PetID", "OwnerID", "PetType", "PetName", "Breed"),
            values=[
                (str(uuid.uuid4()), doug_id, u"Dog", u"Noir", u"Schnoodle"),
                (str(uuid.uuid4()), doug_id, u"Dog", u"Bree", u"Mutt"),
                (str(uuid.uuid4()), doug_id, u"Cat", u"Tom", u"Alley"),
                (str(uuid.uuid4()), john_id, u"Dog", u"Duke", u"GoldenDoodle"),\
                (str(uuid.uuid4()), john_id, u"Dog", u"Sparky", u"Poodle"),
                (str(uuid.uuid4()), john_id, u"Turtle", u"Cuff", u"Box"),
                (str(uuid.uuid4()), john_id, u"Turtle", u"Link", u"Box"),
                (str(uuid.uuid4()), sue_id, u"Cat", u"Cleo", u"Domestic"),
            ],
        )

    print("Inserted data.")


insert_data(instance_id, database_id)

## Go to the Google Cloud Console and verify that the Spanner instance, database, tables, and records were all created. 

## Runs the query passed as an argument

This function runs any SELECT query passed as an argument. 

In [ ]:
def run_query(sql):
  # Instantiate a client.
  spanner_client = spanner.Client(project=project_id)

  # Get a Cloud Spanner instance by ID.
  instance = spanner_client.instance(instance_id)

  # Get a Cloud Spanner database by ID.
  database = instance.database(database_id)

  # Execute a simple SQL statement.
  with database.snapshot() as snapshot:
      results = snapshot.execute_sql(sql)
      for row in results:
        print(row)


## Query with a join

In [ ]:
sql = """SELECT Owners.OwnerID, OwnerName, PetName, PetType, Breed 
         FROM Owners 
         JOIN Pets ON Owners.OwnerID = Pets.OwnerID;"""

run_query(sql)

## Delete all of the data

In [ ]:
def delete_data_with_dml(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    def delete_owners(transaction):
        
        row_ct = transaction.execute_update(
            "DELETE FROM Owners WHERE true = true"
        )

        print("{} record(s) deleted.".format(row_ct))

    database.run_in_transaction(delete_owners)

# Call the function
delete_data_with_dml(instance_id, database_id)


## Delete the database

In [ ]:
def delete_database(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)
    
    database.drop()

    print("{} database dropped".format(database_id))


# Call the function
delete_database(instance_id, database_id)

## Delete the instance

In [ ]:
def delete_spanner_instance(instance_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    instance.delete()

    print("{} instance deleted".format(instance_id))

# Call the function
delete_spanner_instance(instance_id)

## Go to the Console and verify that the Spanner instance was deleted. 